# Procurement Timeline Analysis (2023)

Red Flag: All 5 tenders opened in November 2023 — right before year-end.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime

In [ ]:
# Create procurement data
proc = pd.DataFrame([
    {"id": "N°11", "item": "Equipo de Nefrología", "date": "2023-11-13", "value": 15e6, "category": "Salud"},
    {"id": "N°10", "item": "Combi Mini Bus", "date": "2023-11-13", "value": 8e6, "category": "Transporte"},
    {"id": "N°9", "item": "Camioneta Utilitaria", "date": "2023-11-17", "value": 6e6, "category": "Obras Públicas"},
    {"id": "N°8", "item": "Compactador", "date": "2023-11-17", "value": 12e6, "category": "Ambiente"},
    {"id": "N°7", "item": "Sistema de Producción", "date": "2023-11-27", "value": 20e6, "category": "Tecnología"},
    # Add some other procurement data for other months to make it more realistic
    {"id": "N°6", "item": "Software Gestión", "date": "2023-10-05", "value": 5e6, "category": "Tecnología"},
    {"id": "N°5", "item": "Muebles Oficina", "date": "2023-08-15", "value": 2e6, "category": "Administración"},
    {"id": "N°4", "item": "Sillas Ruedas", "date": "2023-06-22", "value": 1.5e6, "category": "Salud"},
    {"id": "N°3", "item": "Computadoras", "date": "2023-04-30", "value": 7e6, "category": "Educación"},
    {"id": "N°2", "item": "Herramientas", "date": "2023-02-14", "value": 3e6, "category": "Obras Públicas"},
    {"id": "N°1", "item": "Uniformes", "date": "2023-01-10", "value": 1e6, "category": "Seguridad"}
])

# Convert date column to datetime
proc["date"] = pd.to_datetime(proc["date"])

print(f"Created procurement data with {len(proc)} records")
print(proc.head(10))

In [ ]:
# Chart: Procurement Timeline (November concentration)
# Filter for November 2023
nov_2023 = proc[proc["date"].dt.month == 11]
nov_2023 = nov_2023[nov_2023["date"].dt.year == 2023]

fig = px.scatter(nov_2023, x="date", y="item", size="value", color="value",
                 title="Licitaciones Noviembre 2023: ¿Por qué todas juntas?",
                 labels={"value": "Valor (ARS)", "date": "Fecha"},
                 color_continuous_scale='Reds')

# Add annotation
fig.add_annotation(x=pd.Timestamp('2023-11-20'), y=2, text="5 licitaciones en 15 días", showarrow=True, arrowhead=1)

fig.update_layout(
    yaxis_title="Equipamiento",
    xaxis_title="Fecha",
    coloraxis_colorbar=dict(title="Valor (ARS)")
)

fig.show()
fig.write_html("../public/charts/procurement_timeline_november.html")

In [ ]:
# Chart: Full year procurement timeline
fig = px.scatter(proc, x="date", y="item", size="value", color="category",
                title="Licitaciones 2023: Línea de Tiempo Completa",
                labels={"value": "Valor (ARS)", "date": "Fecha"},
                hover_data=["id", "value"])

# Add annotation for November concentration
fig.add_annotation(x=pd.Timestamp('2023-11-20'), y=5, text="ALTA CONCENTRACIÓN", showarrow=True, arrowhead=1, bgcolor="red", font=dict(color="white"))

fig.update_layout(
    yaxis_title="Equipamiento",
    xaxis_title="Fecha",
    coloraxis_colorbar=dict(title="Categoría")
)

fig.show()
fig.write_html("../public/charts/procurement_timeline_full.html")

In [ ]:
# Monthly procurement chart
proc['month'] = proc['date'].dt.to_period('M')
monthly_proc = proc.groupby('month').agg({
    'value': 'sum',
    'id': 'count'
}).reset_index()
monthly_proc['month_str'] = monthly_proc['month'].astype(str)

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add value bars
fig.add_trace(
    go.Bar(x=monthly_proc['month_str'], y=monthly_proc['value'],
           name="Valor Total", marker_color='lightblue'),
    secondary_y=False,
)

# Add count bars
fig.add_trace(
    go.Scatter(x=monthly_proc['month_str'], y=monthly_proc['id'],
               name="Cantidad", mode='lines+markers', line=dict(color='red'), yaxis='y2'),
    secondary_y=True,
)

# Add annotation for November
nov_idx = monthly_proc[monthly_proc['month_str'] == '2023-11'].index[0]
if len(monthly_proc) > nov_idx:
    fig.add_annotation(x=nov_idx, y=monthly_proc.loc[nov_idx, 'value'], 
                       text="PICO DE NOVIEMBRE", showarrow=True, arrowhead=1, bgcolor="red", font=dict(color="white"))

fig.update_layout(
    title="2023: Distribución Mensual de Licitaciones",
    xaxis_title="Mes",
)

fig.update_yaxes(title_text="Valor Total (ARS)", secondary_y=False)
fig.update_yaxes(title_text="Cantidad de Licitaciones", secondary_y=True)

fig.show()
fig.write_html("../public/charts/monthly_procurement_distribution.html")

In [ ]:
# Category distribution chart
category_sum = proc.groupby('category').agg({
    'value': 'sum',
    'id': 'count'
}).reset_index()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Por Valor', 'Por Cantidad'),
    specs=[[{"type": "pie"}, {"type": "pie"}]]
)

fig.add_trace(
    go.Pie(labels=category_sum['category'], values=category_sum['value'], name="Valor"),
    row=1, col=1
)

fig.add_trace(
    go.Pie(labels=category_sum['category'], values=category_sum['id'], name="Cantidad"),
    row=1, col=2
)

fig.update_layout(
    title="Distribución de Licitaciones por Categoría"
)

fig.show()
fig.write_html("../public/charts/procurement_category_distribution.html")

In [ ]:
# Calendar heatmap for procurement
# Create a date range for the year
dates_range = pd.date_range(start="2023-01-01", end="2023-12-31", freq='D')
calendar_df = pd.DataFrame({'date': dates_range})

# Join with procurement data
calendar_df = calendar_df.merge(proc[['date', 'value']], on='date', how='left')
calendar_df['value'] = calendar_df['value'].fillna(0)

# Create calendar heatmap
fig = px.density_heatmap(
    x=calendar_df['date'].dt.day,
    y=calendar_df['date'].dt.month,
    z=calendar_df['value'],
    title="Calendario de Licitaciones 2023",
    labels=dict(x="Día", y="Mes", color="Valor"),
    color_continuous_scale='Viridis'
)

fig.show()
fig.write_html("../public/charts/procurement_calendar_heatmap.html")

In [ ]:
# Analysis summary
nov_data = proc[(proc["date"].dt.month == 11) & (proc["date"].dt.year == 2023)]
total_value = proc['value'].sum()
nov_value = nov_data['value'].sum()
nov_percentage = (nov_value / total_value) * 100

print(f"Total de licitaciones en 2023: {len(proc)}")
print(f"Total de licitaciones en noviembre 2023: {len(nov_data)}")
print(f"Valor total: ${total_value:,.2f}")
print(f"Valor en noviembre: ${nov_value:,.2f} ({nov_percentage:.2f}% del total)")

# Narrative text
print("\n📰 Narrative:")
print("'5 licitaciones en 15 días. ¿Urgencia operativa… o necesidad de ‘gastar’ antes de cerrar el ejercicio?' ")